# 자체 평가지표
* 가지고 있는 제출 파일과 비교

## 데이터 가져오기

In [13]:
import pandas as pd
from tqdm import tqdm

In [14]:
# 판다스 라이브러리에서 과학적 표기 사용 안하기
pd.options.display.float_format = '{:.5f}'.format

In [15]:
# sub24_pycaret12.csv 제출 점수 : 0.1032155541
before_sub = pd.read_csv('../07_결과_출력/sub24_pycaret12.csv')
present_sub = pd.read_csv('../07_결과_출력/sub23_pycaret11.csv')

## 열을 일자, 시간, 구분 열 분해

In [16]:
# 전 체출 파일 열 정리
before_sub[['일자', '시간', '구분']] = before_sub['일자|시간|구분'].str.split(' ').to_list()
before_sub = before_sub.drop('일자|시간|구분', axis=1)
before_sub = before_sub[['일자', '시간', '구분', '공급량']]
# 현재 체출 파일 열 정리
present_sub[['일자', '시간', '구분']] = present_sub['일자|시간|구분'].str.split(' ').to_list()
present_sub = present_sub.drop('일자|시간|구분', axis=1)
present_sub = present_sub[['일자', '시간', '구분', '공급량']]

### 평가 지표 함수화

In [17]:
def evaluation_index(before_sub, present_sub):
    eval_set = pd.DataFrame(['MSE','RMSE', 'MAE', 'R2','NMAE'], columns=['평가지표'])
    for division in tqdm(present_sub['구분'].unique()):
        temp_before_sub = before_sub[before_sub['구분'] == division]
        temp_present_sub = present_sub[present_sub['구분'] == division]

        # 전 제출 값 = 실제값 취급
        # 현제 제출 값 = 예측값
        y = temp_before_sub['공급량']
        pred = temp_present_sub['공급량']
        y_mean = y.mean()

        # MSE
        mse = ( ( y - pred) ** 2 ).sum() / len(pred)
            
        # RMSE
        rmse = mse ** (1/2)
        
        # MAE
        mae = ( abs( y - pred ) ).sum() / len(pred)
        
        # R2
        upper = ( (y - pred) ** 2 ).sum()
        lower = ( (y - y_mean) ** 2 ).sum()
        if lower == 0:
            fraction = 0
        else:
            fraction = upper / lower
        r2 = 1 - fraction
        
        #NMAE(Normalized Mean Absolute Error)-정규화 평균 절대 오차 척도
        nmae = ( abs( y - pred ) / y ).sum() / len(pred)
        
        # 평가지표 리스트
        one_eval_set = [mse, rmse, mae, r2, nmae]
        
        # 평가지표 셋에 담기
        eval_set[division] = one_eval_set

    # 평가지표 평균 구하기
    eval_set['평가지표_평균'] = eval_set.mean(axis=1)

    # 평가지표 평균 출력
    print(eval_set[['평가지표', '평가지표_평균']])
    
    return eval_set

In [18]:
set = evaluation_index(before_sub, present_sub)

100%|██████████| 7/7 [00:00<00:00, 79.76it/s]

   평가지표    평가지표_평균
0   MSE 2190.05414
1  RMSE   40.38706
2   MAE   32.22400
3    R2    0.98544
4  NMAE    0.02240


In [19]:
set

,평가지표,A,B,C,D,E,G,H,평가지표_평균
0,MSE,3026.85151,1658.51080,33.13807,1234.06690,3771.75035,5494.98687,111.07448,2190.05414
1,RMSE,55.01683,40.72482,5.75657,35.12929,61.41458,74.12818,10.53919,40.38706
2,MAE,43.45209,32.71982,4.48817,27.92288,49.48874,59.12160,8.37471,32.22400
3,R2,0.98717,0.99153,0.96654,0.98430,0.98986,0.98757,0.99115,0.98544
4,NMAE,0.02271,0.02097,0.02499,0.02655,0.02162,0.01960,0.02038,0.02240
